In [4]:
import pandas as pd
import mysql.connector
#1

df=pd.read_excel(r"C:\Users\marcelo.diezm_pragma\Documents\reto_airflow\dw-airflow-training\data\raw\educ_uoe_grad05.xlsx")
df2=pd.read_excel(r"C:\Users\marcelo.diezm_pragma\Documents\reto_airflow\dw-airflow-training\data\raw\educ_uoe_grad05.xlsx",sheet_name='Data2')

df['flag']=1
df2['flag']=2


def transformation(df):
    flag=df['flag'][0]
    #escoger la data que necesito
    df=df.iloc[9:,:6].reset_index(drop=True)

    #coger la fila que quiero como columna
    new_header=df.iloc[0]#.apply(pd.to_numeric, errors='coerce')
    #eliminar esa fila
    df = df[1:]
    #poner esa fila como header
    df.columns=new_header

    names_columns=['GEO_TIME','2013','2014','2015','2016', '2017']
    df.columns=names_columns

    df=df[df['GEO_TIME']!=":"]
    df=df[df['GEO_TIME']!="Special value:"]
    # eliminar registros con valores NaN en la columna "x"
    df = df.dropna(subset=['GEO_TIME'])
    # reemplazar ":" por 0 en todo el DataFrame
    df = df.replace(':', 0, regex=True)
    if flag==1:
        value_name='percentage_graduated'
    else:
        value_name='percentage_youth_graduated'
    #derretir el DataFrame en una estructura más larga
    df = pd.melt(df, id_vars=['GEO_TIME'], value_vars=['2013', '2014', '2015', '2016', '2017'], 
                 var_name='year', value_name=value_name)

    # agregar una columna para el porcentaje de jóvenes graduados (asumiendo que es el mismo que el porcentaje general)
    #df['percentage_youth_graduated'] = df['percentage_graduated']

    # pivotear el DataFrame para obtener la estructura deseada
    df = pd.pivot_table(df, values=[value_name], index=['year', 'GEO_TIME'])

    df = df.reset_index()
    return df


def merge_df(df,df2):
    df_final=df.merge(df2,on=["year","GEO_TIME"])
    return df_final


df=transformation(df)
df2=transformation(df2)
df=merge_df(df,df2)


In [6]:
#2
df=pd.read_csv(r"C:\Users\marcelo.diezm_pragma\Documents\reto_airflow\dw-airflow-training\data\raw\grad_5sc.csv", encoding='ISO-8859-1', delimiter=';')

def transformation(df):
    cod_ambito_ambito=df['COD_AMBITO'].str.split("-",n=1,expand=True)
    df['COD_AMBITO']=cod_ambito_ambito[0]
    df['AMBITO']=cod_ambito_ambito[1]
    df['PAIS']='ESPAÑA'
    df['ANIO']= '2016-2017'
    df = df.reindex(columns=['COD_AMBITO','AMBITO','SEXO','EDAD','NUM_EGR_NV1','NUM_EGR_NV2','ANIO'])
    return df

df=transformation(df)

In [7]:
import pandas as pd
#3
df=pd.read_excel(r"C:\Users\marcelo.diezm_pragma\Documents\reto_airflow\dw-airflow-training\data\raw\03003.xlsx")


df=df.iloc[8:,0:5].reset_index(drop=True)
df.columns = ["area_estudio","cantidad_total","trabajando","desempleo","inactivo"]

dfs = []
start = 0
for i in range(len(df)):
    if pd.isnull(df.loc[i, 'trabajando']):
        dfs.append(df.iloc[start:i,:])
        start = i + 1
dfs.append(df.iloc[start:,:])

df_lista=[]
# Iterar a través de la lista
for i in range(len(dfs)):
    # Comprobar la longitud del dataframe
    if len(dfs[i])> 0:
        # Eliminar el dataframe de la lista
        df_lista.append(dfs[i])



#para el primer df
column_general=["ambos_sexos","hombres","mujeres"]
tipo_universidad=["total","publico","privadas"]

for i in range(len(df_lista)):
    if i<=2:
        df_lista[i]['sexo'] = column_general[0]
    if i>=3 and i<=5:
        df_lista[i]['sexo'] = column_general[1]

    if i>=6 and i<=8:
        df_lista[i]['sexo'] = column_general[2]

    tipo = tipo_universidad[i % len(tipo_universidad)]
    df_lista[i]['tipo_universidad'] = tipo
    df_lista[i]['anio'] = 2014
    df_lista[i]['pais'] = 'chile'



#unir los dfs

df_concatenado = pd.concat(df_lista)


# Realizamos el pivoteo
df = pd.melt(df_concatenado, id_vars=['area_estudio', 'cantidad_total', 'sexo', 'tipo_universidad', 'anio', 'pais'],
             value_vars=['trabajando', 'desempleo', 'inactivo'],
             var_name='situacion_laboral', value_name='cantidad')

# Ordenamos el DataFrame según nuestras especificaciones
df = df[['anio', 'pais', 'tipo_universidad', 'area_estudio', 'sexo', 'situacion_laboral', 'cantidad']]

# # Selección de las filas que cumplen la condición
# df_filtered = df.loc[(df['tipo_universidad'] == 'total') & (df['sexo'] == 'ambos_sexos') & (df['area_estudio'].str.strip()=='Total')] 

# # Suma de la columna "cantidad" solo en las filas seleccionadas
# cantidad_total = df_filtered['cantidad'].sum()

# print("La cantidad total es:", cantidad_total)



C:\Users\marcelo.diezm_pragma\AppData\Local\Temp\ipykernel_3732\1740172945.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_lista[i]['sexo'] = column_general[0]
C:\Users\marcelo.diezm_pragma\AppData\Local\Temp\ipykernel_3732\1740172945.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_lista[i]['tipo_universidad'] = tipo
C:\Users\marcelo.diezm_pragma\AppData\Local\Temp\ipykernel_3732\1740172945.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try

In [8]:
df

,anio,pais,tipo_universidad,area_estudio,sexo,situacion_laboral,cantidad
0,2014,chile,total,Total,ambos_sexos,trabajando,149395
1,2014,chile,total,Ciencias sociales y jurídicas,ambos_sexos,trabajando,77778
2,2014,chile,total,Ingeniería y arquitectura,ambos_sexos,trabajando,35923
3,2014,chile,total,Artes y humanidades,ambos_sexos,trabajando,8522
4,2014,chile,total,Ciencias de la salud,ambos_sexos,trabajando,19295
...,...,...,...,...,...,...,...
157,2014,chile,privadas,Ciencias sociales y jurídicas,mujeres,inactivo,595
158,2014,chile,privadas,Ingeniería y arquitectura,mujeres,inactivo,150
159,2014,chile,privadas,Artes y humanidades,mujeres,inactivo,37
160,2014,chile,privadas,Ciencias de la salud,mujeres,inactivo,199


In [12]:
df1 = df_concatenado[(df_concatenado['tipo_universidad'] != 'total') & (df_concatenado['area_estudio'].str.strip() != 'Total')]

In [ ]:
df_filtered = df.loc[(df['tipo_universidad'] == 'total') & (df['sexo'] == 'ambos_sexos') & df_filtered[df_filtered['area_estudio'].str.strip()=='Total']  ]
df_filtered

In [ ]:
df_filtered

In [ ]:
df_filtered = df.loc[(df['tipo_universidad'] == 'total') & df_filtered[df_filtered['area_estudio'].str.strip()=='Total']  ]
df_filtered

In [ ]:
df